In [1]:
from test import EigenData
import numpy as np
import scipy.io
import h5py
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from math import sqrt
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import torch
%matplotlib inline

In [2]:
data = EigenData()
eigenworms = data.get_eigenworms('EigenWorms.mat')

footage = data.get_footage('20150814-All-PNAS2011-DataStitched .mat')
newdata = torch.rand(33600,5)
for k in footage.keys():
    print(k, footage[k].shape)
    arr1=footage[k]

    arr2=arr1[:5,]

    r = data.reconstruct(footage[k])
    print('-->', r.shape)
    dataset1 = arr2.transpose()

    newdata = np.concatenate((newdata,dataset1),axis=0)
newdata = newdata[33600:,]
print(newdata.shape)

b (5, 33600)
(100, 5) (5, 33600)
--> (100, 33600)
c (5, 33600)
(100, 5) (5, 33600)
--> (100, 33600)
d (5, 33600)
(100, 5) (5, 33600)
--> (100, 33600)
e (5, 33600)
(100, 5) (5, 33600)
--> (100, 33600)
f (5, 33600)
(100, 5) (5, 33600)
--> (100, 33600)
g (6, 33600)
(100, 6) (6, 33600)
--> (100, 33600)
h (5, 33600)
(100, 5) (5, 33600)
--> (100, 33600)
i (6, 33600)
(100, 6) (6, 33600)
--> (100, 33600)
j (5, 33600)
(100, 5) (5, 33600)
--> (100, 33600)
k (5, 33600)
(100, 5) (5, 33600)
--> (100, 33600)
l (5, 33600)
(100, 5) (5, 33600)
--> (100, 33600)
m (5, 33600)
(100, 5) (5, 33600)
--> (100, 33600)
(403200, 5)


In [3]:
first_5_eigenworms = eigenworms[0:5,:]
eig_worm_0 = eigenworms[0,:]
eig_worm_1 = eigenworms[1,:]
eig_worm_2 = eigenworms[2,:]
eig_worm_3 = eigenworms[3,:]
eig_worm_4 = eigenworms[4,:]

In [4]:
data1=newdata[~np.isnan(newdata).any(axis=1)]
data1 = data1.astype('float32')
print(data1.shape)
print(len(data1))

(386422, 5)
386422


In [5]:
scaler = MinMaxScaler(feature_range=(0, 1))
data1 = scaler.fit_transform(data1)

In [6]:
def create_dataset(dataset, look_back=5):
    dataX, dataY = [], []
    for i in range(len(dataset) - look_back):
        a = dataset[i:(i + look_back)]
        dataX.append(a)
        dataY.append(dataset[i + look_back])
    return np.array(dataX), np.array(dataY)

In [7]:
data_X, data_Y = create_dataset(data1)

print(data_X.shape)
print(data_Y.shape)

(386417, 5, 5)
(386417, 5)


In [8]:
train_size = int(len(data_X) * 0.8)
test_size = len(data_X) - train_size
train_X = data_X[:train_size]
train_Y = data_Y[:train_size]
test_X = data_X[train_size:]
test_Y = data_Y[train_size:]

print(train_X.shape)
print(train_Y.shape)
print(test_X.shape)
print(test_Y.shape)

(309133, 5, 5)
(309133, 5)
(77284, 5, 5)
(77284, 5)


In [9]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.layers import Input,LSTM,Dropout,Dense



# create and fit the LSTM network

model = keras.Sequential()
model.add(LSTM(100,input_shape=(5,5),return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(100,input_shape=(5,5),return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(100,input_shape=(5,5),return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(100,input_shape=(5,5),return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(5,activation='linear'))
model.compile(optimizer='rmsprop',loss='mean_squared_error',metrics=[tf.keras.metrics.RootMeanSquaredError()])

history = model.fit(train_X, train_Y, epochs=100, batch_size=1024, verbose=2, validation_data=(test_X, test_Y))

Epoch 1/100
302/302 - 49s - loss: 0.0147 - root_mean_squared_error: 0.1211 - val_loss: 0.0056 - val_root_mean_squared_error: 0.0750
Epoch 2/100
302/302 - 41s - loss: 0.0062 - root_mean_squared_error: 0.0789 - val_loss: 0.0034 - val_root_mean_squared_error: 0.0587
Epoch 3/100
302/302 - 41s - loss: 0.0042 - root_mean_squared_error: 0.0645 - val_loss: 0.0020 - val_root_mean_squared_error: 0.0450
Epoch 4/100
302/302 - 40s - loss: 0.0028 - root_mean_squared_error: 0.0533 - val_loss: 0.0010 - val_root_mean_squared_error: 0.0321
Epoch 5/100
302/302 - 39s - loss: 0.0021 - root_mean_squared_error: 0.0456 - val_loss: 5.4463e-04 - val_root_mean_squared_error: 0.0233
Epoch 6/100
302/302 - 40s - loss: 0.0017 - root_mean_squared_error: 0.0409 - val_loss: 4.1585e-04 - val_root_mean_squared_error: 0.0204
Epoch 7/100
302/302 - 39s - loss: 0.0014 - root_mean_squared_error: 0.0379 - val_loss: 6.8593e-04 - val_root_mean_squared_error: 0.0262
Epoch 8/100
302/302 - 39s - loss: 0.0013 - root_mean_squared_err

Epoch 60/100
302/302 - 42s - loss: 3.8094e-04 - root_mean_squared_error: 0.0195 - val_loss: 1.3744e-04 - val_root_mean_squared_error: 0.0117
Epoch 61/100
302/302 - 41s - loss: 3.7930e-04 - root_mean_squared_error: 0.0195 - val_loss: 9.4667e-05 - val_root_mean_squared_error: 0.0097
Epoch 62/100
302/302 - 41s - loss: 3.7841e-04 - root_mean_squared_error: 0.0195 - val_loss: 1.2292e-04 - val_root_mean_squared_error: 0.0111
Epoch 63/100
302/302 - 42s - loss: 3.7692e-04 - root_mean_squared_error: 0.0194 - val_loss: 9.2679e-05 - val_root_mean_squared_error: 0.0096
Epoch 64/100
302/302 - 42s - loss: 3.7566e-04 - root_mean_squared_error: 0.0194 - val_loss: 9.5798e-05 - val_root_mean_squared_error: 0.0098
Epoch 65/100
302/302 - 42s - loss: 3.7511e-04 - root_mean_squared_error: 0.0194 - val_loss: 9.5184e-05 - val_root_mean_squared_error: 0.0098
Epoch 66/100
302/302 - 42s - loss: 3.7331e-04 - root_mean_squared_error: 0.0193 - val_loss: 1.3919e-04 - val_root_mean_squared_error: 0.0118
Epoch 67/100


In [10]:
testPredict = model.predict(test_X)

In [11]:
print(testPredict.shape)
print(test_Y.shape)

(77284, 5)
(77284, 5)


In [12]:
newtestPredict = scaler.inverse_transform(testPredict)
newtest_Y = scaler.inverse_transform(test_Y)

In [13]:
testScore = sqrt(mean_squared_error(newtest_Y, newtestPredict))
print('Test Score: %.5f RMSE' % (testScore))

Test Score: 0.32686 RMSE
